In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/jfk.csv')

In [4]:
dataset.shape

(8757, 1049)

In [5]:
dataset.head(3)

,Date,Hour,1,10,100,101,102,106,107,108,...,91_lag_3,92_lag_3,93_lag_3,94_lag_3,95_lag_3,96_lag_3,97_lag_3,98_lag_3,99_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,0,0,0,0,0,...,1.0,1.0,0.0,1.0,6.0,0.0,1.0,0.0,0.0,6.0
1,2018-01-01,4,0,3,0,0,1,0,0,1,...,4.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0
2,2018-01-01,5,0,4,0,0,1,2,3,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0


In [6]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

777

In [7]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 272)

In [8]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

258

In [9]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [10]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 272)
(2190, 272)


In [11]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 258])
torch.Size([2190, 13])
torch.Size([2190, 258])


In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
#         train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq_x, train_seq_y ,train_label))
    return inout_seq

In [13]:
bptt = 24

In [14]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [15]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape, train_inout_seq[0][2].shape

(torch.Size([24, 13]), torch.Size([24, 258]), torch.Size([24, 258]))

In [16]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [17]:
class LSTM(nn.Module):
    def __init__(self, feat_size=1, hidden_layer_size=100, network_size=1, layers=1, communities=10):
        super().__init__()
        
        # aggregation
        self.attachment_matrix = torch.nn.Parameter(torch.randn(network_size,communities))
        self.attachment_matrix.requires_grad = True
        
        
        self.hidden_layer_size = hidden_layer_size
        
        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                    torch.zeros(layers,1,self.hidden_layer_size))
        
        lstm_input = communities + feat_size
        self.lstm = nn.LSTM(input_size=lstm_input, hidden_size=hidden_layer_size, num_layers=layers)

        #disaggregation
#         self.linear_1 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.linear_2 = nn.Linear(hidden_layer_size, network_size)


    def forward(self, input_seq, feat):
        
        w = F.softmax(self.attachment_matrix, dim=1)
        x = torch.matmul(input_seq, self.attachment_matrix)
        x = torch.cat((x,feat),axis=1)

        
        lstm_out, self.hidden_cell = self.lstm(x.view(len(input_seq) ,1, -1), self.hidden_cell)
        
        predictions = self.linear_2(lstm_out.view(len(input_seq), -1))
#         predictions = F.relu(predictions)
#         predictions = self.linear_2(predictions)
        
        return predictions

In [18]:
def evaluate(model):
    model.eval()
    prediction = []
    with torch.no_grad():
        for feat,seq, labels in test_inout_seq:
            model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                            torch.zeros(layers, 1, model.hidden_layer_size))
            prediction.append(model(seq,feat)[-1])

    y_test_ = torch.stack([labels[-1] for feat,seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
    y_pred_ = torch.stack(prediction).detach().cpu().numpy()

    r2 = r2_score(y_test_, y_pred_, multioutput='variance_weighted')
    rmse = mean_squared_error(y_test_, y_pred_)
    mae = mean_absolute_error(y_test_, y_pred_)
#     print("r2: ",r2)
    return (r2, rmse, mae)

In [19]:
layers = 1
communities = 20
network_size = len(targetColumns)
feat_size = len(features_cols)

model = LSTM(feat_size = feat_size, hidden_layer_size=100,
             network_size=network_size, layers=layers,
            communities=communities).to(device)

loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

In [20]:
epochs = 250


for i in range(epochs):
    model.train()
    for feat,seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq, feat)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        
    scheduler.step()
    if i%10 == 1:
        r2, rmse, mae = evaluate(model)
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 1.23411751 r2: 0.416 rmse: 4.983 mae: 1.251
epoch:  11 loss: 1.25594020 r2: 0.484 rmse: 4.397 mae: 1.155
epoch:  21 loss: 1.26784170 r2: 0.469 rmse: 4.533 mae: 1.176
epoch:  31 loss: 1.17462993 r2: 0.506 rmse: 4.211 mae: 1.127
epoch:  41 loss: 1.18800199 r2: 0.513 rmse: 4.152 mae: 1.122
epoch:  51 loss: 1.18325198 r2: 0.505 rmse: 4.225 mae: 1.126
epoch:  61 loss: 1.15604198 r2: 0.518 rmse: 4.110 mae: 1.109
epoch:  71 loss: 1.18215728 r2: 0.514 rmse: 4.145 mae: 1.108
epoch:  81 loss: 1.16961443 r2: 0.514 rmse: 4.146 mae: 1.112
epoch:  91 loss: 1.15114784 r2: 0.510 rmse: 4.179 mae: 1.111
epoch: 101 loss: 1.14751267 r2: 0.511 rmse: 4.173 mae: 1.111
epoch: 111 loss: 1.14076138 r2: 0.510 rmse: 4.179 mae: 1.111
epoch: 121 loss: 1.14463723 r2: 0.510 rmse: 4.182 mae: 1.110
epoch: 131 loss: 1.13794863 r2: 0.508 rmse: 4.200 mae: 1.111
epoch: 141 loss: 1.13818395 r2: 0.508 rmse: 4.200 mae: 1.110
epoch: 151 loss: 1.13970613 r2: 0.508 rmse: 4.195 mae: 1.109
epoch: 161 loss: 1.13945

In [21]:
evaluate(model)

(0.5026385642873215, 4.2423506, 1.1048625)

In [22]:
attachment = torch.argmax(F.softmax(model.attachment_matrix, dim=1), dim=1).detach().cpu().numpy()
community_assignment = dict(zip(targetColumns, attachment))
community_assignment

{'1': 8,
 '10': 6,
 '100': 10,
 '101': 3,
 '102': 15,
 '106': 7,
 '107': 3,
 '108': 7,
 '109': 7,
 '11': 19,
 '110': 11,
 '111': 19,
 '112': 4,
 '113': 19,
 '114': 5,
 '115': 18,
 '116': 2,
 '117': 13,
 '118': 13,
 '119': 7,
 '12': 9,
 '120': 6,
 '121': 11,
 '122': 6,
 '123': 15,
 '124': 2,
 '125': 2,
 '126': 10,
 '127': 0,
 '128': 18,
 '129': 0,
 '13': 12,
 '130': 18,
 '131': 15,
 '133': 15,
 '134': 7,
 '135': 11,
 '136': 14,
 '137': 0,
 '138': 8,
 '139': 16,
 '14': 2,
 '140': 12,
 '141': 16,
 '142': 18,
 '143': 2,
 '144': 18,
 '145': 3,
 '146': 2,
 '147': 11,
 '148': 15,
 '149': 10,
 '15': 19,
 '150': 7,
 '151': 14,
 '152': 1,
 '153': 2,
 '154': 6,
 '155': 5,
 '156': 7,
 '157': 7,
 '158': 13,
 '159': 7,
 '16': 11,
 '160': 19,
 '161': 16,
 '162': 16,
 '163': 3,
 '164': 0,
 '165': 3,
 '166': 19,
 '167': 1,
 '168': 10,
 '169': 14,
 '17': 2,
 '170': 4,
 '171': 7,
 '172': 8,
 '173': 3,
 '174': 10,
 '175': 19,
 '176': 7,
 '177': 2,
 '178': 3,
 '179': 18,
 '18': 11,
 '180': 2,
 '181': 18,
 

In [23]:
# 2 linear
# 0.47

In [24]:
# 1 linear
# 0.51

In [25]:
# 1 linear + RELU
# 0.50

In [ ]:
# 1 linear bptt = 24
# 0.52

In [ ]:
#  TODO
# other hubs
# ensemble model